In [8]:
import pandas as pd
import numpy as np
from functools import  singledispatch
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from importlib import reload
import eval_funcs 
import plotting_funcs as pf 
import hickle as hkl
import ALA_helpers 
import non_ala_helpers as nah
from scipy.stats import trim_mean

In [9]:
full_E_decomp_dict_multi_muts = hkl.load("multi_muts_E_decomp_neut_all.hkl")
single_res_df = nah.df_from_hkl(full_E_decomp_dict_multi_muts)
try :
	full_E_decomp_dict_multi_muts_cleaned= single_res_df.drop(columns=["gb5opt5d", "gb5opt1"])
except KeyError:
    full_E_decomp_dict_multi_muts_cleaned = single_res_df
wild_type_avg = full_E_decomp_dict_multi_muts_cleaned.iloc[[0,3]].mean(axis=0)
df_cleaned_multi =  full_E_decomp_dict_multi_muts_cleaned.iloc[4:]

In [10]:
full_E_decomp_dict_multi_muts_cleaned.iloc[[0,3]].std(axis=0)

gb6opt1       1.412375
gb6mbondi2    3.141888
gb5mbondi2    2.460661
gb6opt5d      0.348674
dtype: float64

In [11]:
df_cleaned_multi

,gb6opt1,gb6mbondi2,gb5mbondi2,gb6opt5d
Beta,-53.7997,-69.0876,-57.8429,-53.6648
E484KN501Y,-56.5621,-69.1214,-57.8232,-52.4799
Gamma,-56.8007,-67.2388,-57.8126,-57.8762
K417NE484K,-52.5619,-64.6043,-52.1028,-51.8304
K417TE484K,-58.0985,-70.6288,-56.7670,-57.0735


In [12]:
full_E_decomp_dict_single_res = hkl.load("single_res_E_decomp_all.hkl")
single_res_df = nah.df_from_hkl(full_E_decomp_dict_single_res)
df_cleaned_single_res = single_res_df.drop(columns=["gb5opt5d", "gb5opt1"])
df_exp_data = pd.read_csv("single_mut_effects.csv")
ddg_num_single = df_cleaned_single_res - wild_type_avg
sims_done = [  mut[0:3] + nah.three_letter_amino[mut[3:]]  for mut in list(ddg_num_single.index)]
cleaned_exp_dms_data = eval_funcs.relevant_exp_results(sims_done, df_exp_data).drop(columns=['site_RBD', 'site_SARS2', 'wildtype', 'mutant', 
       'mutation_RBD', 'bind_lib1', 'bind_lib2', 'bind_avg',
       'expr_lib1', 'expr_lib2', 'expr_avg'])
new_idx_arry = [mut[1:4] + nah.amino_acids[mut[4]] for mut in cleaned_exp_dms_data["mutation"]]
cleaned_exp_dms_data_cp = cleaned_exp_dms_data.copy()
cleaned_exp_dms_data_cp.index = new_idx_arry
cleaned_exp_dms_rdy = cleaned_exp_dms_data_cp.drop(columns=["mutation"])
merged_result = pd.merge(ddg_num_single, cleaned_exp_dms_rdy, left_index=True, right_index=True)
dddg_df = ddg_num_single.sub(cleaned_exp_dms_rdy["del_del_G"], axis=0)

In [13]:
dddg_df

,gb5mbondi2,gb6mbondi2,gb6opt1,gb6opt5d
417ASP,1.202824,-0.958876,-2.775126,-4.282976
439GLU,1.118589,-2.805911,-2.715361,-5.992511
446CYS,0.986544,-1.048256,-1.530906,-3.197556
453PRO,-0.129609,-4.069109,-5.774359,-7.149109
455ASP,3.720335,4.039235,3.944285,5.024035
475TRP,7.448260,6.675660,6.466410,4.450660
486ASP,-0.171205,-1.318105,-1.051755,-0.676505
487ILE,7.426672,6.655172,8.795022,7.514872
489SER,13.499391,17.418391,17.531441,18.249391
493ASP,6.172386,3.677186,4.817736,3.418186


In [14]:
trimmed_mean_value = trim_mean((dddg_df**2), proportiontocut=0.2)
RMSE_single = ( (dddg_df**2).mean() ) ** (1/2)

In [15]:
RMSE_single

gb5mbondi2    7.356367
gb6mbondi2    8.007912
gb6opt1       8.981578
gb6opt5d      8.886162
dtype: float64

In [16]:
dddg_df.mean()

gb5mbondi2    5.163932
gb6mbondi2    3.994638
gb6opt1       4.514759
gb6opt5d      3.172885
dtype: float64

In [17]:
RMSE_single

gb5mbondi2    7.356367
gb6mbondi2    8.007912
gb6opt1       8.981578
gb6opt5d      8.886162
dtype: float64

In [18]:
wild_type_avg

gb6opt1      -53.34560
gb6mbondi2   -65.34965
gb5mbondi2   -57.83065
gb6opt5d     -50.53935
dtype: float64

In [19]:
df_cleaned_multi

,gb6opt1,gb6mbondi2,gb5mbondi2,gb6opt5d
Beta,-53.7997,-69.0876,-57.8429,-53.6648
E484KN501Y,-56.5621,-69.1214,-57.8232,-52.4799
Gamma,-56.8007,-67.2388,-57.8126,-57.8762
K417NE484K,-52.5619,-64.6043,-52.1028,-51.8304
K417TE484K,-58.0985,-70.6288,-56.7670,-57.0735


In [20]:
ddg_sims = df_cleaned_multi -wild_type_avg
hardcoded_ddg_spr = pd.Series(index=df_cleaned_multi.index.to_list(), data=np.array([0.79, 1.62, 1.03 , -0.72, -0.43]))

In [21]:
hardcoded_ddg_spr

Beta          0.79
E484KN501Y    1.62
Gamma         1.03
K417NE484K   -0.72
K417TE484K   -0.43
dtype: float64

In [22]:
ddg_sims.sub( -hardcoded_ddg_spr, axis=0)

,gb6opt1,gb6mbondi2,gb5mbondi2,gb6opt5d
Beta,0.3359,-2.94795,0.77775,-2.33545
E484KN501Y,-1.5965,-2.15175,1.62745,-0.32055
Gamma,-2.4251,-0.85915,1.04805,-6.30685
K417NE484K,0.0637,0.02535,5.00785,-2.01105
K417TE484K,-5.1829,-5.70915,0.63365,-6.96415


In [23]:
ddg_multi_df = ddg_sims.sub( -hardcoded_ddg_spr, axis=0)
updated_order = ddg_multi_df[[ 'gb5mbondi2','gb6mbondi2', 'gb6opt1',  'gb6opt5d']].copy()
updated_col_names = ['GB5 Mbondi2', 'GB66 Mbondi2', 'GB66 OPT1', 'GB66 OPT5d', ]
latex_ready_multi = updated_order.rename(columns=dict(zip([ 'gb5mbondi2','gb6mbondi2', 'gb6opt1',  'gb6opt5d'],  updated_col_names)))

In [24]:
(latex_ready_multi**2).mean()**(1/2)

GB5 Mbondi2     2.442620
GB66 Mbondi2    3.054623
GB66 OPT1       2.661175
GB66 OPT5d      4.424413
dtype: float64

In [25]:
ALA_helpers._df_to_latex_cust(latex_ready_multi)

\begin{tabular}{lrrrr}
\toprule
 & GB5 Mbondi2 & GB66 Mbondi2 & GB66 OPT1 & GB66 OPT5d \\
\midrule
Beta & 0.78 & \textbf{-2.95} & 0.34 & -2.34 \\
E484KN501Y & 1.63 & \textbf{-2.15} & -1.60 & -0.32 \\
Gamma & 1.05 & -0.86 & -2.43 & \textbf{-6.31} \\
K417NE484K & 5.01 & 0.03 & 0.06 & \textbf{-2.01} \\
K417TE484K & 0.63 & -5.71 & -5.18 & \textbf{-6.96} \\
\bottomrule
\end{tabular}

